<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Clustering the Predictions of the Metr-LA dataset</h2>
</center>

---

In this notebook the predictions of the *STGNN* on the *PeMS-Bay* dataset are clustered in order to obtain events to explain such as congestions or free-flows.

Firstly, a distance matrix is obtained for each predicted instance of the datasets in order to compute the spatio-temporal and speed distance among all nodes in the prediction. The distance matrix is obtained throught the method explained in *Revealing the day-to-day regularity of urban congestion patterns with
3d speed maps* <a name="cite_paper"></a>[<sup>[1]</sup>](#note_paper)

Finally, the predictions are clustered through *DBSCAN*, while considering the distance matrix as a dissimilarity measure.

For more detailed informations about the used functions, look into the corresponding docstrings inside the python files, inside the `src` folder.

---
<small>

<a name="note_paper"></a>[1] 
C. Lopez et al. “Revealing the day-to-day regularity of urban congestion patterns with
3d speed maps”. In: *Scientific Reports, 7(1):14029*, September 2017. ISSN:
2045-2322. DOI: 10.1038/s41598-017-14237-8. URL: https://doi.org/10.1038/s41598-017-14237-8.
</small>

In [2]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [3]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [4]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [5]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# 1 Loading the Data
In this section the data is loaded.

In [6]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'pems-bay')

In [7]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [8]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_pems_bay.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_pems_bay.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the model in evaluation mode.
spatial_temporal_gnn.eval();

In [9]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_pems_bay.csv'),
    has_header=False)

In [10]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [11]:
import os
import numpy as np
from src.spatial_temporal_gnn.prediction import predict


# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'predicted', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'predicted', 'y_train.npy'))
x_val = np.load(os.path.join(BASE_DATA_DIR, 'predicted', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'predicted', 'y_val.npy'))
x_test = np.load(os.path.join(BASE_DATA_DIR, 'predicted', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'predicted', 'y_test.npy'))

# Get the time information of the train, validation and test sets.
x_train_time = np.load(
    os.path.join(BASE_DATA_DIR, 'predicted', 'x_train_time.npy'))
y_train_time = np.load(
    os.path.join(BASE_DATA_DIR, 'predicted', 'y_train_time.npy'))
x_val_time = np.load(
    os.path.join(BASE_DATA_DIR, 'predicted', 'x_val_time.npy'))
y_val_time = np.load(
    os.path.join(BASE_DATA_DIR, 'predicted', 'y_val_time.npy'))
x_test_time = np.load(
    os.path.join(BASE_DATA_DIR, 'predicted', 'x_test_time.npy'))
y_test_time = np.load(
    os.path.join(BASE_DATA_DIR, 'predicted', 'y_test_time.npy'))

The predictions are turned into km/h

In [12]:
# Turn the results in kilometers per hour.
from src.utils.config import MPH_TO_KMH_FACTOR


y_train = y_train * MPH_TO_KMH_FACTOR
y_val = y_val * MPH_TO_KMH_FACTOR
y_test = y_test * MPH_TO_KMH_FACTOR

In [13]:
_, n_timesteps, n_nodes, _ = y_train.shape

# 2 Distance matrix
The distance matrix $M$ used as a metric of dissimilarity in DBSCAN is computed separately for each instance and is composed of:
* A *spatial distance matrix* $M_d$;
* A *temporal distance matrix* $M_t$;
* A *speed distance matrix* $M_s$

The matrices $M_d$, $M_t$ and $M_s$ are each scaled through *min-max scaling* between $0$ and $1$ and summed together in order to obtain $M$ as follows: 
$$M = W \cdot M_s + M_d + M_t$$
where the speed distance is overweighted by multiplying $M_s$ by a factor $W \geq 1$ because the speed variable is expected to play a predominant role during the clustering process. $M$ is next normalized again between $0$ and $1$ through min-max scaling.

## 2.1 Spatial Distance Matrix
The *spatial distance matrix* $M_d$ is an $(N \cdot T') \times (N \cdot T')$ matrix derived from the adjacency matrix of the traffic network $A \in \mathbb{R}^{N \times N}$ and describing the spatial distant of each nodes regardless of their timestep.

In [14]:
from src.explanation.clustering.clustering import (
    get_adjacency_distance_matrix)

adj_distance_matrix = get_adjacency_distance_matrix(adj_matrix, n_timesteps)

In [15]:
print(f'Shape of the Adjacency Distance Matrix: {adj_distance_matrix.shape}')

Shape of the Adjacency Distance Matrix: (3900, 3900)


# 2.2 Temporal Distance Matrix
The *temporal distance matrix* $M_t$ is an $(N \cdot T') \times (N \cdot T')$ matrix describing the temporal distance of the nodes at each timestep.

In [16]:
from src.explanation.clustering.clustering import (
    get_temporal_distance_matrix)

temporal_distance_matrix = get_temporal_distance_matrix(n_nodes, n_timesteps)

In [17]:
print('Shape of the Temporal Distance Matrix:',
      f'{temporal_distance_matrix.shape}')

Shape of the Temporal Distance Matrix: (3900, 3900)


# 2.3 Speed Distance Matrix
The *speed distance matrix* $M_s$ is an $(N \cdot T') \times (N \cdot T')$ matrix describing the speed distance of the nodes at each timestep.

It is computed for each instance separately, before DBSCAN is performed. The value $W$, overweighting it is set as $3$.

# 3 Clustering Function
Next, the clustering technique *DBSCAN* is used on the distance matrix $M$. It identifies whether each node of the predicted traffic network at a given time, specified by an index in $M$, is part of a distinct traffic cluster or categorized as noise.

## 3.1 Grid Search
Grid search is performed on a portion of the train dataset in order to find the best set of hyperparameters of DBSCAN:
* `eps`
* `min_samples`

The results are evaluated in terms of *Within Cluster Variance*, *Cluster Dissimilarity* and *noise ratio*.

* **Within Cluster Variance:**
    $$WCV = \frac{1}{\sum_{i = 1}^N n_i} \frac{\sum_{i = 1}^N n_i \sigma^2_i}{\sigma^2} $$
    with $N$ the number of clusters, $n_i$ the nodes of the $i^{th}$ cluster in the predicted network and $\sigma_i^2$ the speed variance among its nodes. $\sigma^2$ is the variance among all nodes of the prediction.

* **Cluster Dissimilarity:**
    $$ CD = \frac{\sum_{i = 1}^N \sum_{k = i + 1}^N \sqrt{n_i \cdot n_k} \cdot |\mu_i - \mu_k|}{\sum_{i = 1}^N \sum_{k = i + 1}^N \sqrt{n_i \cdot n_k}} $$
    with $\mu_i$ the mean speed value of cluster $i$.

* **Noise Ratio:** It measures the ratio of the nodes classified as outliers by DBSCAN in a predicted traffic network.

In [18]:
from src.explanation.clustering.evaluation import apply_grid_search

# Apply the grid search on a subset of the training set.
apply_grid_search(
    instances=y_train[:200],
    eps_list=[.1, .15, .2, .25, .3, .35, .4, .45, .5],
    min_samples_list=[5, 7, 10, 12, 15, 17, 20],
    adj_distance_matrix=adj_distance_matrix,
    temporal_distance_matrix=temporal_distance_matrix)

eps: 0.1 min_samples: 5
	Within-Cluster Variance: 0.984 Connected Cluster Dissimilarity: 4.2 Noise points ratio: 0.95

eps: 0.1 min_samples: 7
	Within-Cluster Variance: 0.997 Connected Cluster Dissimilarity: 4.06 Noise points ratio: 0.991

eps: 0.1 min_samples: 10
	Within-Cluster Variance: 1 Connected Cluster Dissimilarity: 0.827 Noise points ratio: 0.999

eps: 0.1 min_samples: 12
	Within-Cluster Variance: 1 Connected Cluster Dissimilarity: 0 Noise points ratio: 1

eps: 0.1 min_samples: 15
	Within-Cluster Variance: 1 Connected Cluster Dissimilarity: 0 Noise points ratio: 1

eps: 0.1 min_samples: 17
	Within-Cluster Variance: 1 Connected Cluster Dissimilarity: 0 Noise points ratio: 1

eps: 0.1 min_samples: 20
	Within-Cluster Variance: 1 Connected Cluster Dissimilarity: 0 Noise points ratio: 1

eps: 0.15 min_samples: 5
	Within-Cluster Variance: 0.839 Connected Cluster Dissimilarity: 6.12 Noise points ratio: 0.677

eps: 0.15 min_samples: 7
	Within-Cluster Variance: 0.964 Connected Cluster 

The best hyperparameters are expressed below.

In [19]:
# Set the best parameters based on the results of the grid search.

EPS = .35
MIN_SAMPLES = 5

## 3.2 Clustering and Saving the Results
In this section the clustering of the predictions of the train, validation and test datasets is computed using the selected hyperparameters.

The results are evaluated in terms of *Within Cluster Variance*, *Cluster Dissimilarity* and *noise ratio* on the test set.

In [21]:
from src.explanation.clustering.evaluation import get_dataset_clustering_scores

(avg_within_cluster_variance, avg_connected_cluster_dissimilarity,
 avg_noise_ratio) = get_dataset_clustering_scores(
     y_test, adj_distance_matrix, temporal_distance_matrix, EPS, MIN_SAMPLES)

print(
    'Within-Cluster Variance on the test set:',
    f'{avg_within_cluster_variance:.3g}',
    'Connected Cluster Dissimilarity on the test set:',
    f'{avg_connected_cluster_dissimilarity:.3g}',
    'Noise points ratio on the test set:', f'{avg_noise_ratio:.3g}')

Within-Cluster Variance on the test set: 0.159 Connected Cluster Dissimilarity on the test set: 9.61 Noise points ratio on the test set: 0.00601


In [22]:
import os

DATA_DIR = os.path.join('..', 'data', 'pems-bay', 'explainable')

In [23]:
from numpy import save
from src.explanation.clustering.clustering import (
    get_dataset_for_explainability)

os.makedirs(DATA_DIR, exist_ok=True)

(x_train_expl, y_train_expl,
 x_train_time_expl, y_train_time_expl) = get_dataset_for_explainability(
    x_train,
    y_train,
    x_train_time,
    y_train_time,
    EPS,
    MIN_SAMPLES,
    adj_distance_matrix,
    temporal_distance_matrix,
    total_samples=1_000)
save(os.path.join(DATA_DIR, 'x_train.npy'), x_train_expl)
save(os.path.join(DATA_DIR, 'y_train.npy'), y_train_expl)
save(os.path.join(DATA_DIR, 'x_train_time.npy'), x_train_time_expl)
save(os.path.join(DATA_DIR, 'y_train_time.npy'), y_train_time_expl)

(x_val_expl, y_val_expl,
 x_val_time_expl, y_val_time_expl) = get_dataset_for_explainability(
    x_val,
    y_val,
    x_val_time,
    y_val_time,
    EPS,
    MIN_SAMPLES,
    adj_distance_matrix,
    temporal_distance_matrix,
    total_samples=200)
save(os.path.join(DATA_DIR, 'x_val.npy'), x_val_expl)
save(os.path.join(DATA_DIR, 'y_val.npy'), y_val_expl)
save(os.path.join(DATA_DIR, 'x_val_time.npy'), x_val_time_expl)
save(os.path.join(DATA_DIR, 'y_val_time.npy'), y_val_time_expl)

(x_test_expl, y_test_expl,
 x_test_time_expl, y_test_time_expl) = get_dataset_for_explainability(
    x_test,
    y_test,
    x_test_time,
    y_test_time,
    EPS,
    MIN_SAMPLES,
    adj_distance_matrix,
    temporal_distance_matrix,
    total_samples=300)
save(os.path.join(DATA_DIR, 'x_test.npy'), x_test_expl)
save(os.path.join(DATA_DIR, 'y_test.npy'), y_test_expl)
save(os.path.join(DATA_DIR, 'x_test_time.npy'), x_test_time_expl)
save(os.path.join(DATA_DIR, 'y_test_time.npy'), y_test_time_expl)

In [24]:
print('Train dataset for explainability shapes:',
      x_train_expl.shape, y_train_expl.shape)
print('Validation dataset for explainability shapes:',
      x_val_expl.shape, y_val_expl.shape)
print('Test dataset for explainability shapes:',
      x_test_expl.shape, y_test_expl.shape)

Train dataset for explainability shapes: (999, 12, 325, 9) (999, 12, 325, 1)
Validation dataset for explainability shapes: (198, 12, 325, 9) (198, 12, 325, 1)
Test dataset for explainability shapes: (300, 12, 325, 9) (300, 12, 325, 1)
